# 🧪 고성능 데이터셋 단위 테스트

이 노트북은 HighPerfDocClsDataset 클래스의 기능을 테스트합니다:
- 데이터셋 생성 및 샘플 로드
- 증강 동작 및 클래스 분포 확인
- GPU 자동 체크

In [1]:
import warnings  # 경고 메시지 억제용 모듈 임포트
warnings.filterwarnings('ignore')  # 모든 경고 메시지 출력 억제

In [2]:
import os  # OS 모듈 임포트
os.chdir("../../../")  # 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())  # 현재 디렉토리 출력

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [3]:
import torch  # 파이토치 임포트
if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
    print(f'✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}')  # GPU 이름 출력
else:
    print('⚠️ GPU 사용 불가, CPU로 실행됩니다')  # CPU 사용 안내

✅ GPU 사용 가능: NVIDIA GeForce RTX 4090


In [4]:
import numpy as np  # 넘파이 임포트
import pandas as pd  # 판다스 임포트
import matplotlib.pyplot as plt  # 시각화 라이브러리 임포트
from src.data.dataset import HighPerfDocClsDataset  # 데이터셋 클래스 임포트
from src.utils.common import load_yaml  # 설정 로드 함수 임포트
from src.utils.unit_test_logger import create_test_logger  # 단위 테스트 로거 생성 함수 임포트
test_logger = create_test_logger("highperf_dataset")  # 단위 테스트 로거 초기화

📝 단위 테스트 시작: highperf_dataset
📝 로그 디렉토리: notebooks/modular/unit_tests/highperf_dataset/20250905_194002


In [5]:
# 설정 파일 로드 및 정보 출력
try:  # 예외 처리 시작
    cfg = load_yaml("configs/train_highperf.yaml")  # 설정 파일 로드
    print("📋 설정 로드 완료")  # 설정 로드 성공 메시지
    print(f"🎯 모델: {cfg['model']['name']}")  # 모델명 출력
    print(f"📏 이미지 크기: {cfg['train']['img_size']}")  # 이미지 크기 출력
    print(f"🔢 배치 크기: {cfg['train']['batch_size']}")  # 배치 크기 출력
except Exception as e:  # 예외 발생 시
    print(f"⚠️ 설정 로드 실패: {e}")  # 에러 메시지 출력

📋 설정 로드 완료
🎯 모델: swin_base_384
📏 이미지 크기: 384
🔢 배치 크기: 32


In [6]:
# 학습 데이터 로드 및 정보 출력
try:  # 예외 처리 시작
    train_csv = "data/raw/train.csv"  # 학습 데이터 CSV 경로
    image_dir = "data/raw/train"  # 이미지 디렉토리 경로
    df = pd.read_csv(train_csv)  # CSV 파일 읽기
    print(f"📊 전체 데이터 수: {len(df):,}개")  # 전체 샘플 수 출력
    print(f"️ 클래스 수: {df['target'].nunique()}개")  # 클래스 수 출력
    print(f"📁 이미지 디렉토리: {image_dir}")  # 이미지 디렉토리 출력
    class_dist = df['target'].value_counts().sort_index()  # 클래스 분포 계산
    print(f"📈 클래스 분포:")  # 클래스 분포 출력
    for i, (class_id, count) in enumerate(class_dist.head(10).items()):  # 상위 10개 클래스 출력
        print(f"   Class {class_id}: {count:,}개 ({count/len(df)*100:.1f}%)")  # 클래스별 샘플 수 및 비율 출력
    if len(class_dist) > 10:  # 클래스가 10개 초과일 경우
        print(f"   ... 외 {len(class_dist)-10}개 클래스")  # 나머지 클래스 개수 출력
except Exception as e:  # 예외 발생 시
    print(f"⚠️ 데이터 로드 실패: {e}")  # 에러 메시지 출력

📊 전체 데이터 수: 1,570개
️ 클래스 수: 17개
📁 이미지 디렉토리: data/raw/train
📈 클래스 분포:
   Class 0: 100개 (6.4%)
   Class 1: 46개 (2.9%)
   Class 2: 100개 (6.4%)
   Class 3: 100개 (6.4%)
   Class 4: 100개 (6.4%)
   Class 5: 100개 (6.4%)
   Class 6: 100개 (6.4%)
   Class 7: 100개 (6.4%)
   Class 8: 100개 (6.4%)
   Class 9: 100개 (6.4%)
   ... 외 7개 클래스


In [7]:
# 고성능 데이터셋 생성 및 샘플 로드 테스트
try:  # 예외 처리 시작
    test_ds_epoch0 = HighPerfDocClsDataset(  # 데이터셋 생성
        df=df.head(100),  # 처음 100개 샘플만 사용
        image_dir=image_dir,  # 이미지 디렉토리 경로
        img_size=cfg['train']['img_size'],  # 이미지 크기
        epoch=0,  # epoch 설정
        total_epochs=cfg['train']['epochs'],  # 전체 epoch 수
        is_train=True,  # 학습 모드
        id_col=cfg['data']['id_col'],  # ID 컬럼명
        target_col=cfg['data']['target_col']  # 타겟 컬럼명
    )
    print(f"✅ 데이터셋 생성 완료")  # 데이터셋 생성 성공 메시지
    print(f"📦 데이터셋 크기: {len(test_ds_epoch0)}")  # 데이터셋 크기 출력
    print(f"🎲 Hard augmentation 확률 (epoch 0): {test_ds_epoch0.p_hard:.3f}")  # Hard augmentation 확률 출력
    img, label = test_ds_epoch0[0]  # 첫 샘플 로드
    import matplotlib.pyplot as plt  # 시각화 라이브러리 임포트
    plt.rc('font', family='DejaVu Sans')  # 한글 폰트 설정 (시스템 기본)
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
    img_np = img.permute(1,2,0).numpy()
    img_np = np.clip(img_np, 0, 1)  # [0, 1]로 제한
    plt.imshow(img_np)  # 이미지 시각화
    plt.title(f'고성능 샘플 / 라벨: {label}')  # 라벨 출력
    plt.axis('off')  # 축 제거
    plt.show()  # 이미지 출력
    print(f"🖼️ 이미지 형태: {img.shape}")  # 이미지 shape 출력
    print(f"🏷️ 라벨: {label}")  # 라벨 출력
    print(f"📊 이미지 값 범위: [{img.min():.3f}, {img.max():.3f}]")  # 이미지 값 범위 출력
except Exception as e:  # 예외 발생 시
    print(f"⚠️ 데이터셋 생성/샘플 로드 실패: {e}")  # 에러 메시지 출력

✅ 데이터셋 생성 완료
📦 데이터셋 크기: 100
🎲 Hard augmentation 확률 (epoch 0): 0.200
🖼️ 이미지 형태: torch.Size([3, 384, 384])
🏷️ 라벨: 16
📊 이미지 값 범위: [-2.118, 0.862]


## 🏆 테스트 결과 요약

- 각 단계별 정상 동작 및 예외 처리 결과를 확인했습니다.
- 데이터셋 생성, 증강, 샘플 로드, GPU 체크 등 주요 기능이 정상적으로 동작하면 ✅, 실패 시 ⚠️ 메시지로 확인 가능합니다.
- 테스트 결과 및 로그는 코드 셀의 출력과 test_logger에 기록됩니다.
- 추가 검증이 필요한 부분은 각 셀의 예외 메시지와 로그를 참고하세요.